In [2]:
import numpy as np
import plotly.graph_objects as go

import DataRetriever as dr

RETRIEVER = dr.DataRetriever()
CON_ATTRIBUTES = RETRIEVER.get_attributes(file_name='consuming_attributes.pkl')
DATA = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')[CON_ATTRIBUTES].sum(axis=1)

TRAIN_SIZE = int(DATA.size * 0.8)

TRAIN, TEST = DATA[0:TRAIN_SIZE].to_numpy(), DATA[TRAIN_SIZE: DATA.size].to_numpy()
TRAIN_INDEX, TEST_INDEX = list(range(0, len(TRAIN))), list(range(0, len(TEST)))

In [ ]:
def fourierExtrapolation(data: np.array, number_of_predictions: int, model_harmonics: int) -> np.array:
    data_size = data.size
    sample_index = np.arange(0, data_size)
    linear_trend = np.polyfit(sample_index, data, 1)
    data_detrended = data - linear_trend[0] * sample_index

    frequency_domain = np.fft.fft(data_detrended)
    frequencies = np.fft.fftfreq(data_size)

    indexes = list(range(data_size))
    indexes.sort(key=lambda idx: np.absolute(frequencies[idx]))  # sort indices by frequency, lower -> higher

    sample_index = np.arange(0, data_size + number_of_predictions)
    restored_sig = np.zeros(sample_index.size)

    for i in indexes[:1 + model_harmonics * 2]:
        amplitude = np.absolute(frequency_domain[i]) / data_size  # amplitude
        phase = np.angle(frequency_domain[i])  # phase
        restored_sig += amplitude * np.cos(2 * np.pi * frequencies[i] * sample_index + phase)
    return restored_sig + linear_trend[0] * sample_index


def main():
    x = TRAIN
    n_predict = TEST.size
    n_harmonics = 1_000

    predictions = fourierExtrapolation(data=x,
                                       number_of_predictions=n_predict,
                                       model_harmonics=n_harmonics)

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=np.arange(DATA),
                             y=predictions,
                             name='Prediction',
                             mode='lines'))

    fig.add_trace(go.Scatter(x=np.arange(DATA),
                             y=DATA,
                             name='Test Data',
                             mode='lines'))

    fig.show()


if __name__ == "__main__":
    main()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [4]:
0.125 * (1 / ((2 * np.pi) ** 2 + 1))

0.003088065378982205